# 手撕Transformer

本文件将按照"Attention is all you need"论文中的结构图对涉及到的组件进行逐一实现，架构图如下所示：

![Transformer架构图](resource/transformer_frame.png)

## 位置编码Positional Encoding

因为Transformer架构不同于串行处理的RNN和LSTM，所以需要额外引入位置编码给输入序列注入位置信息。

### 绝对正弦-余弦位置编码（原版编码方式）

位置编码计算公式如下图所示：

![正余弦位置编码公式](resource/sincosPE.png)

公式中的参数解释如下：

1. p表示输入序列分词后某个token所在的位置索引

2. i表示当前所属维度，分奇偶进行成对计算

3. d表示维度，此处的维度与分词得到token后，token经过embedding层所得得到的向量表示的维度（位置编码要与序列编码相加，所以维度要相同）

In [1]:
import torch
import torch.nn as nn
import math


class SincosPE(nn.Module):
    # d_model表示维度，max_seq_length提前创建长度为80的位置编码（固定不改变）
    def __init__(self, d_model, max_seq_length=80):
        super().__init__()
        
        self.d_model = d_model
        # 创建PE矩阵
        pe = torch.zeros(max_seq_length, d_model)
        for pos in range(max_seq_length):
            for i in range(0, d_model, 2):
                # 实现公式对应的位置编码计算
                pe[pos, i] = math.sin(pos / (100000) ** ((2 * i) / d_model))
                pe[pos, i + 1] = math.cos(pos / (100000) ** ((2 * i) / d_model))

        # 此时的PE矩阵是二维的，需要升维满足batch训练的shape
        pe = pe.unsqueeze(0)        # [1, max_seq_length, d_model]

        # 此时的pe在输入序列确定的时候就已经是确认不再发生改变的状态了
        # 参数形式nn.Parameter存储优化器会尝试更新，不符合要求
        # 以类内变量self.pe=pe存储，CPU/GPU切换时会增加手动处理的复杂
        self.register_buffer('pe', pe)      # torch进行自动管理，且不进行梯度计算和更新

    def forward(self, x):
        # 此时传入的x是输入序列经过分词嵌入后得到的
        # 原论文提及需要进行放缩，理由是避免与位置编码相加时数值过小导致位置信号的稀释
        x = x * math.sqrt(self.d_model)     # 根号d是经验数值

        # 因为pe是固定80创建的，避免影响到eos_token等特殊字符，此处只对有效位置进行加和
        seq_len = x.size(1)     # x的shape是[batch, seq_len, d_model]
        x = x + self.pe[:, :seq_len]
        return x

In [ ]:
PE = SincosPE(8)
PE.pe

### 旋转位置编码RoPE

## 多头注意力机制Multihead Attention

此步骤即是对Transformer的核心机制Self-Attention的实现，多头则是处理不同序列跨度的多个独立的注意力头。Self-Attention的一个输出向量计算如下图所示：

![Self-Attention推导示意图](resource/self_attention.pngneed_delete)

Multi-head Attention结构如下图所示：

![Multi-head Attention结构示意图](resource/multi-head.pngneed_delete)

### 重点参数及原因解析

在论文"Attention is all you need"的原文对Multi-head Attention的表述是这样的：

Instead of performing a single attention function with dmodel-dimensional keys, values and queries, we found it beneficial to linearly project the queries, keys and values h times with different, learned linear projections to dk, dk and dv dimensions, respectively. On each of these projected versions of queries, keys and values we then perform the attention function in parallel, yielding dv-dimensional output values. These are concatenated and once again projected, resulting in the final values。

上述话所阐述的关键在于，原本Self-Attention中所有维度的信息集中在一个注意力头回被迫“平均”学习所有类型的关系，导致任何关系都无法学的好。**因此，将原维度平均拆分成h个更小的维度，每个维度单独学习特定类型的特征或关系效果将更好**

**上述描述所需的参数如下**

1. **d_model：** 经过分词及词嵌入层后所产生的维度
2. **d_k：** 每个注意力头所分配的维度
3. **heads：** 注意力头的数量

多头注意力的计算方式也与单头存在不同，如下所示原文计算公式：

![Multi-head Attention计算公式](resource/multi-head_attention.png)

公式中每个头的attention的计算都是原始的QKV矩阵乘上对应头的参数矩阵$W_i$，去获取每个头专属的独立空间的QKV，再去计算attention score，此时的/{}